In [25]:
# Initial Setup:
import pandas as pd
import numpy as np
import re
from datetime import datetime

url = 'https://www.sharkattackfile.net/spreadsheets/GSAF5.xls'
df = pd.read_excel(url)

In [ ]:
# Preview of raw DataFrame:
print("head:")
print(df.head())

print("\ninfo:")
df.info()

print("\ndescribe:")
print(df.describe())

print("\ncolumns:")
print(df.columns.tolist())

In [ ]:
# DataFrame cleaning preparation:

# 1. Dropping unneeded columns and duplicates:
columns_to_drop = ["Source", "Location", "Injury", "Name", "pdf", "href formula",
                   "href", "Case Number", "Case Number.1", "original order",
                   "Unnamed: 21", "Unnamed: 22", "Species "]

df = df.drop(columns=columns_to_drop).drop_duplicates().reset_index(drop=True)

# 2. Filter rows with Year > 1800 and remove unneeded 'Types':
df = df[df["Year"] > 1800]

# 3. Types of shark attacks to exclude:
undesired_types = ["Questionable", "Boat", "Provoked", "Provoked ", "?",
                   "Unverified", "Under investigation", "Unconfirmed"]

df = df[~df["Type"].isin(undesired_types)]

# 4. Renaming and reformatting columns:
df.columns = [col.strip().replace(" ", "_").replace(".", "").lower() for col in df.columns]
df.rename(columns={'unnamed:_11': 'fatal'}, inplace=True)

# 5. Creating a copy of the original DataFrame for further manipulation
df_copy = df.copy()

# Check:
print(df.columns.tolist())

In [ ]:
# Cleaning 'fatal' column:
value_map = {'n': 'n', 'y': 'y'}

df_copy['fatal'] = df_copy['fatal'].str.strip().str.lower().map(value_map)

# Calculate mode:
fatal_mode = df_copy['fatal'].mode()[0]

# Replace NaNs with 'n':
df_copy['fatal'] = df_copy['fatal'].fillna(fatal_mode)

# Check:
df_copy['fatal'].value_counts(dropna=False)

In [ ]:
# Cleaning 'sex' column:
df_copy['sex'] = df_copy['sex'].str.strip().str.lower()

# Replace wrong values with 'unknown':
invalid_entries = ['lli', 'm x 2', 'n', '.']
for entry in invalid_entries:
    df_copy['sex'] = df_copy['sex'].replace(entry, 'unknown')

# Replace missing values with 'unknown':
df_copy['sex'].fillna('unknown')

# Check:
df_copy['sex'].value_counts(dropna=False)

In [ ]:
#Cleaning 'type' column:

# Remove 'invalid' values:
df_copy = df_copy[df_copy['type'] != 'invalid']

# Replace 'NaN' with 'unknown':
df_copy['type'].fillna('unknown')

# Check:
df_copy['type'].value_counts(dropna=False)

In [31]:
# unique_values = df['time'].unique()
# print(unique_values)

In [ ]:
# 1. Convert 'time' column to strings (to avoid issues with NaN when splitting)
df_copy['time'] = df_copy['time'].astype(str)

# 2. Function to validate if time is in the correct "hhmm" format (e.g., "16h30")
def validate_time_format(time_str):
    # Regex pattern for valid time formats like "16h00", "01h50", etc.
    pattern = r'^\d{2}h\d{2}$'
    if re.match(pattern, time_str):
        return time_str
    else:
        return None  # Invalid format, will replace with mean later

# 3. Apply the validation function and keep only valid times
df_copy['time'] = df_copy['time'].apply(lambda x: x.replace('h', '') if validate_time_format(x) else None)

# 4. Convert the 'time_numeric' to numeric, replacing invalid parsing with None
df_copy['time'] = pd.to_numeric(df_copy['time'], errors='coerce')

# 5. Calculate the mean time, ignoring NaNs
mean_time = df_copy['time'].mean()

# 6. Fill NaN values (both invalid format and actual NaNs) with the calculated mean time
df_copy['time'].fillna(round(mean_time), inplace=True)

# 7. Convert all values in 'time_numeric' to integers
df_copy['time'] = df_copy['time'].astype(int)

# Output the updated dataframe
print(df)



In [ ]:
# Cleaning 'age' column:

# Converting 'age' to numeric, setting errors to NaN:
df_copy['age'] = pd.to_numeric(df_copy['age'], errors='coerce')

# Calculating the mean age, excluding NaNs:
age_mean = df_copy['age'].mean()
print(f"Mean age: {age_mean}")

# Filling missing 'age' values with the calculated mean:
df_copy['age'] = df_copy['age'].fillna(age_mean)

# Rounding 'age' values to nearest whole number:
df_copy['age'] = df_copy['age'].round(0)

# Check:
df_copy['age'].value_counts(dropna=False)

In [ ]:
def reformat_date_adjusted(date_str):
    # Ensure the input is a string; early return for empty or NaN-like strings
    date_str = str(date_str).strip()
    if not date_str or date_str.lower() == 'nan':
        return "invalid-date"
    # Normalize the date string
    date_str = re.sub(r'^Reported\s+', '', date_str, flags=re.IGNORECASE)
    date_str = re.sub(r'\s*-\s*|\s+', ' ', date_str)  # Convert '-' to ' ' and collapse multiple spaces
    # Try parsing the date with different formats
    try:
        # Detect format based on separators and content
        if re.search(r'\d{2}\s\w+\s\d{4}', date_str):  # DD MMM YYYY or DD MMMM YYYY
            for fmt in ("%d %b %Y", "%d %B %Y"):  # Try both abbreviated and full month name formats
                try:
                    parsed_date = datetime.strptime(date_str, fmt)
                    return parsed_date.strftime("%m-%Y")
                except ValueError:
                    pass  # If one format fails, try the next
        elif re.search(r'\w+\s\d{4}', date_str):  # MMM YYYY or MMMM YYYY
            for fmt in ("%b %Y", "%B %Y"):  # Try both abbreviated and full month name formats
                try:
                    parsed_date = datetime.strptime(date_str, fmt)
                    return parsed_date.strftime("%m-%Y")
                except ValueError:
                    pass
        elif re.match(r'\d{4}$', date_str):  # YYYY only
            return datetime.strptime(date_str, "%Y").strftime("%Y") + "-00"
    except Exception as e:
        print(f"Error processing date '{date_str}': {e}")
    return "Unknown"
# Application of the function:
df_copy['date'] = df_copy['date'].astype(str)
df_copy['date'] = df_copy['date'].apply(reformat_date_adjusted)
# Diagnostic check to review the transformation results
print(df_copy['date'].value_counts(dropna=False))

In [35]:
# Seasonality function and new seasonality column

def get_seasonality(formatted_date):
    try:
        month = int(formatted_date.split('-')[0])
        if month in [12, 1, 2]:
            return "Winter"
        elif month in [3, 4, 5]:
            return "Spring"
        elif month in [6, 7, 8]:
            return "Summer"
        elif month in [9, 10, 11]:
            return "Autumn"
        else:
            return "Unknown"
    except:
        return "Unknown"

df_copy['seasonality'] = df_copy['date'].apply(get_seasonality)

In [ ]:
# Seasonality stats
valid_seasons_df = df_copy[df_copy['seasonality'] != "Unknown"]
season_counts = valid_seasons_df['seasonality'].value_counts()
total_count = len(valid_seasons_df)
season_percentage = (season_counts / total_count) * 100

print(season_percentage)

In [ ]:
# Replace 'NaN' with 'unknown':
df_copy['activity'].fillna('unknown', inplace=True)

# Check:
df_copy['activity'].value_counts(dropna=False)


In [ ]:
# Replace NaNs in the State column
df["state"].fillna("Unknown")

In [ ]:
# Join 'country' and 'state' into 'location' to preserve data and enhance precision.
location = df_copy['country'].astype(str) + ', ' + df_copy['state'].astype(str)
df_copy = df_copy.join(location.rename('location'))
df_copy

In [ ]:
# Subsequent overview of v0.5:
# Cell to be (re)moved or updated as we refine our analysis and finalize other KPIs.

print("head:")
print(df_copy.head())

print("\ninfo:")
df_copy.info()

print("\ndescribe:")
print(df_copy.describe())

print("\ncolumns:")
print(df_copy.columns.tolist())

print("\nmissing values:")
print(df_copy.isnull().sum()[df_copy.isnull().sum() > 0])

# Pending Cleaning Columns: 'date', 'year', 'activity', 'country', 'state'.
# Note on Geo Data: Considering merging 'country' and 'state' into 'location' to preserve data and enhance precision.

# Upcoming in v0.6:
# - Cleaning for 'date', 'year', 'type' and their integration.
# - Decision pending on creating a 'location' column and its integration.

# Uniform Commentary: Please keep documentation clear and accessible.